In [2]:
import pandas as pd
import random
import pandas as pd
import numpy as np
import datetime as dt
import itertools
import csv
import time
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df=pd.read_csv('~/spotify_session_data/log_mini.csv')
track_df=pd.read_csv('~/spotify_session_data/tf_mini.csv')

In [3]:
df

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,t_64f3743c-f624-46bb-a579-0f3f9a07a123,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167875,0_0eaeef5d-25e9-4429-bd55-af15d3604c9f,16,20,t_360910e8-2a84-42b0-baf1-59abcf96a1f2,False,False,False,True,0,1,...,0,0,0,False,13,2018-07-15,True,user_collection,trackdone,trackdone
167876,0_0eaeef5d-25e9-4429-bd55-af15d3604c9f,17,20,t_aa2fff77-9b0a-4fa3-a685-ecef50310e8a,False,False,False,True,0,1,...,0,0,0,False,13,2018-07-15,True,user_collection,trackdone,trackdone
167877,0_0eaeef5d-25e9-4429-bd55-af15d3604c9f,18,20,t_f673e1b7-4ebe-4fc1-ac24-a9f25de70381,False,False,False,True,0,1,...,0,0,0,False,13,2018-07-15,True,user_collection,trackdone,trackdone
167878,0_0eaeef5d-25e9-4429-bd55-af15d3604c9f,19,20,t_e172e8e7-7161-42a9-acb0-d606346c8f87,False,False,False,True,0,1,...,0,0,0,False,13,2018-07-15,True,user_collection,trackdone,trackdone


In [4]:
track_df

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_a540e552-16d4-42f8-a185-232bd650ea7d,109.706673,1950,99.975414,0.458040,0.519497,0.504949,0.399767,7.511880,0.817709,...,4,0.935512,-0.033284,-0.411896,-0.028580,0.349438,0.832467,-0.213871,-0.299464,-0.675907
1,t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,...,3,0.359675,0.145703,-0.850372,0.123860,0.746904,0.371803,-0.420558,-0.213120,-0.525795
2,t_0614ecd3-a7d5-40a1-816e-156d5872a467,160.839996,1951,99.602549,0.812884,0.425890,0.508280,0.491625,8.368670,0.358813,...,4,0.726769,0.021720,-0.743634,0.333247,0.568447,0.411094,-0.187749,-0.387599,-0.433496
3,t_070a63a0-744a-434e-9913-a97b02926a29,175.399994,1951,99.665018,0.396854,0.400934,0.359990,0.552227,5.967346,0.514585,...,4,0.859075,0.039143,-0.267555,-0.051825,0.106173,0.614825,-0.111419,-0.265953,-0.542753
4,t_d6990e17-9c31-4b01-8559-47d9ce476df1,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,4,0.562343,0.131931,-0.292523,-0.174819,-0.034422,0.717229,-0.016239,-0.392694,-0.455496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50699,t_402930af-4174-47ec-b1fd-593d93597624,184.686798,2018,99.315966,0.584765,0.521544,0.515087,0.653140,7.684220,0.336433,...,4,0.542063,-0.196001,0.301727,0.238880,-0.391421,0.016690,0.247235,-0.399387,-0.192473
50700,t_e5f9a069-a893-452e-ab21-49b4eaebfbd0,251.813324,2018,99.918573,0.406680,0.565200,0.693126,0.844861,11.176841,0.709085,...,4,0.472353,-0.545160,0.271596,0.274377,0.043951,-0.322946,0.150802,0.159378,0.384336
50701,t_3983306d-13b4-4027-9391-7236ca93d2bf,157.520004,2018,98.517692,0.001279,0.414721,0.341769,0.463543,5.405471,0.975503,...,4,0.766519,0.112592,0.368523,-0.466950,-0.468494,0.640088,0.050771,-0.258999,0.258766
50702,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,4,0.058505,-0.855291,0.365487,0.273034,0.108294,-0.206204,0.007847,-0.408226,0.143629


In [5]:
import pandas as pd
# Perform an inner merge based on 'track_id' and 'track_id_clean'
merged_df = pd.merge(track_df, df, left_on='track_id', right_on='track_id_clean', how='inner')
merged_df

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,t_a540e552-16d4-42f8-a185-232bd650ea7d,109.706673,1950,99.975414,0.458040,0.519497,0.504949,0.399767,7.511880,0.817709,...,0,0,0,True,10,2018-07-15,True,user_collection,trackdone,trackdone
1,t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,...,0,0,0,False,23,2018-07-14,True,radio,trackdone,trackdone
2,t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,...,0,0,0,False,23,2018-07-14,True,radio,backbtn,fwdbtn
3,t_0614ecd3-a7d5-40a1-816e-156d5872a467,160.839996,1951,99.602549,0.812884,0.425890,0.508280,0.491625,8.368670,0.358813,...,1,0,0,False,22,2018-07-14,True,user_collection,trackdone,trackdone
4,t_070a63a0-744a-434e-9913-a97b02926a29,175.399994,1951,99.665018,0.396854,0.400934,0.359990,0.552227,5.967346,0.514585,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,endplay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167875,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,13,2018-07-15,False,user_collection,fwdbtn,fwdbtn
167876,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,2,2018-07-15,False,user_collection,fwdbtn,fwdbtn
167877,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,True,22,2018-07-14,True,editorial_playlist,fwdbtn,trackdone
167878,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,17,2018-07-15,False,user_collection,trackdone,trackdone


In [6]:
merged_df['session_id'].value_counts()

session_id
0_004f4627-4bb0-4c79-b93f-f85d494108ac    20
0_02ea1de9-186e-4cb6-9509-215a434d0024    20
0_085887ec-0b1f-433e-8cb5-7481e780c1ed    20
0_07d4ceef-49b3-475b-b4b6-9cbc8237ff10    20
0_05dd7cbb-3fde-4daa-b461-223ca5dc9d75    20
                                          ..
0_044a391f-37c6-422b-9cd1-7a3ec54cc19b    10
0_09fcb11e-26df-4ac8-a4c0-eaf18ebbb9e8    10
0_003d0b7d-8c68-49b4-b0c0-49e501978523    10
0_0d687ef6-77bc-4707-b18a-329828bbd02f    10
0_0e4a49d8-688d-4958-94db-5cb56adc3090    10
Name: count, Length: 10000, dtype: int64

In [7]:
merged_df.columns

Index(['track_id', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mechanism', 'mode', 'organism', 'speechiness',
       'tempo', 'time_signature', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7', 'session_id', 'session_position', 'session_length',
       'track_id_clean', 'skip_1', 'skip_2', 'skip_3', 'not_skipped',
       'context_switch', 'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'date', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end'],
 

In [8]:
sample_df = merged_df[merged_df['session_length'] == 20].reset_index(drop=True)

In [9]:
sample_df

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,t_070a63a0-744a-434e-9913-a97b02926a29,175.399994,1951,99.665018,0.396854,0.400934,0.359990,0.552227,5.967346,0.514585,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,endplay
1,t_d6990e17-9c31-4b01-8559-47d9ce476df1,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,0,0,0,False,22,2018-07-14,True,user_collection,trackdone,trackdone
2,t_d6990e17-9c31-4b01-8559-47d9ce476df1,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,1,0,0,True,13,2018-07-15,True,user_collection,fwdbtn,fwdbtn
3,t_d6990e17-9c31-4b01-8559-47d9ce476df1,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,0,0,0,False,0,2018-07-15,True,editorial_playlist,fwdbtn,trackdone
4,t_d6990e17-9c31-4b01-8559-47d9ce476df1,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,0,0,0,False,16,2018-07-15,True,catalog,clickrow,trackdone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101315,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,19,2018-07-14,True,catalog,clickrow,trackdone
101316,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,13,2018-07-15,False,user_collection,fwdbtn,fwdbtn
101317,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,2,2018-07-15,False,user_collection,fwdbtn,fwdbtn
101318,t_74eb6e99-210b-440c-8d7b-4db6617d1c80,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,True,22,2018-07-14,True,editorial_playlist,fwdbtn,trackdone


In [10]:
sample_df['session_id'].value_counts()

session_id
0_091c94f6-e76b-4b3e-8366-5d9d3c3e2fef    20
0_06eeb361-eaf3-4623-96b4-556a2486a6df    20
0_07eb705f-7a53-41c8-b93d-f2f6349977b1    20
0_07cd73e7-50b3-4043-a579-e44f770939f6    20
0_07b43604-9721-4165-9b14-3064070de466    20
                                          ..
0_0ce7bf2b-1687-46e1-8620-410694b0a79c    20
0_0336648a-c8b4-4de5-92ed-23821bbb75c4    20
0_0af9d3e1-08c8-4744-8258-85ab64545a80    20
0_0525bf5b-6dc9-4390-bcb7-e3c5fa354c1d    20
0_044d1e7d-fdaa-46f7-acfc-672740c193ed    20
Name: count, Length: 5066, dtype: int64

In [11]:
sample_df['session_id'].nunique()

5066

In [12]:
sample_df['session_id'] = LabelEncoder().fit_transform(sample_df['session_id'])
sample_df['track_id'] = LabelEncoder().fit_transform(sample_df['track_id'])

In [13]:
sample_df

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,980,175.399994,1951,99.665018,0.396854,0.400934,0.359990,0.552227,5.967346,0.514585,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,endplay
1,29544,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,0,0,0,False,22,2018-07-14,True,user_collection,trackdone,trackdone
2,29544,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,1,0,0,True,13,2018-07-15,True,user_collection,fwdbtn,fwdbtn
3,29544,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,0,0,0,False,0,2018-07-15,True,editorial_playlist,fwdbtn,trackdone
4,29544,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,...,0,0,0,False,16,2018-07-15,True,catalog,clickrow,trackdone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101315,16266,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,19,2018-07-14,True,catalog,clickrow,trackdone
101316,16266,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,13,2018-07-15,False,user_collection,fwdbtn,fwdbtn
101317,16266,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,False,2,2018-07-15,False,user_collection,fwdbtn,fwdbtn
101318,16266,129.105392,2018,99.902866,0.139452,0.688375,0.733720,0.850959,10.778521,0.666146,...,0,0,0,True,22,2018-07-14,True,editorial_playlist,fwdbtn,trackdone


In [14]:
columns_to_convert = ['skip_1', 'skip_2', 'skip_3','not_skipped']

# Convert the selected columns to binary (0/1)
sample_df[columns_to_convert] = sample_df[columns_to_convert].astype(int)

# Now, the selected columns are converted to binary (0/1)
print(sample_df)

        track_id    duration  release_year  us_popularity_estimate  \
0            980  175.399994          1951               99.665018   
1          29544  369.600006          1951               99.991764   
2          29544  369.600006          1951               99.991764   
3          29544  369.600006          1951               99.991764   
4          29544  369.600006          1951               99.991764   
...          ...         ...           ...                     ...   
101315     16266  129.105392          2018               99.902866   
101316     16266  129.105392          2018               99.902866   
101317     16266  129.105392          2018               99.902866   
101318     16266  129.105392          2018               99.902866   
101319     24174  213.689392          2018               99.961653   

        acousticness  beat_strength  bounciness  danceability  dyn_range_mean  \
0           0.396854       0.400934    0.359990      0.552227        5.967346 

In [15]:
for column in columns_to_convert:
    print(f"Column: {column}")
    print(f"Data Type: {sample_df[column].dtype}")
    print(f"Unique Values: {sample_df[column].unique()}")

Column: skip_1
Data Type: int64
Unique Values: [0 1]
Column: skip_2
Data Type: int64
Unique Values: [0 1]
Column: skip_3
Data Type: int64
Unique Values: [1 0]
Column: not_skipped
Data Type: int64
Unique Values: [0 1]


In [16]:
historic_users = []
for i, u in enumerate(sample_df['session_id'].unique()):
    temp = sample_df[sample_df['session_id'] == u]
    temp = temp.sort_values('session_position').reset_index()
    temp.drop('index', axis = 1, inplace = True)
    historic_users.append(temp)

In [17]:
historic_users[0]

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,21974,174.533325,1973,99.894184,0.131766,0.443350,0.422686,0.596957,6.717382,0.660066,...,0,0,0,False,20,2018-07-14,True,catalog,clickrow,trackdone
1,980,175.399994,1951,99.665018,0.396854,0.400934,0.359990,0.552227,5.967346,0.514585,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,endplay
2,12827,153.240005,1959,99.993917,0.717835,0.337198,0.333590,0.392218,6.105763,0.628898,...,1,0,0,False,20,2018-07-14,True,catalog,clickrow,endplay
3,4176,219.026672,1967,99.991521,0.325646,0.370907,0.372939,0.395721,6.492118,0.472904,...,0,0,0,False,20,2018-07-14,True,catalog,clickrow,trackdone
4,17890,149.373337,1963,99.991131,0.660822,0.576118,0.578336,0.550206,8.383670,0.853673,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,fwdbtn
5,18654,173.039993,1965,99.984579,0.253322,0.379395,0.380625,0.521229,6.551623,0.486795,...,0,0,0,False,20,2018-07-14,True,catalog,fwdbtn,trackdone
6,12649,134.386673,1964,99.981306,0.360272,0.382199,0.395234,0.389514,6.777518,0.624586,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,trackdone
7,20837,217.093323,1984,99.989182,0.010279,0.468185,0.447077,0.682708,6.927204,0.636635,...,1,0,0,False,20,2018-07-14,True,catalog,trackdone,endplay
8,850,148.533325,1963,99.925367,0.650558,0.438553,0.389850,0.521903,6.141509,0.370431,...,0,0,0,False,20,2018-07-14,True,catalog,clickrow,endplay
9,1424,134.133331,1963,99.903337,0.184336,0.426840,0.364696,0.340783,5.760113,0.416226,...,0,0,0,False,20,2018-07-14,True,catalog,clickrow,trackdone


In [39]:
def sample_histo(self, user_histo, action_ratio = 0.25, max_samp_by_user = 10,  max_state = 10, max_action = 5,
                 nb_states = [], nb_actions = []):
        
        '''
        For a given historic, make one or multiple sampling.
        If no optional argument given for nb_states and nb_actions, then the sampling
        is random and each sample can have differents size for action and state.
        To normalize sampling we need to give list of the numbers of states and actions
        to be sampled.

        Parameters
        ----------
        user_histo :  DataFrame
                          historic of user
        delimiter :       string, optional
                          delimiter for the csv
        action_ratio :    float, optional
                          ratio form which song tracks in history will be selected
        max_samp_by_user: int, optional
                          Nulber max of sample to make by user
        max_state :       int, optional
                          Number max of song tracks to take for the 'state' column
        max_action :      int, optional
                          Number max of song tracks to take for the 'action' action
        nb_states :       array(int), optional
                          Numbers of song tracks to be taken for each sample made on user's historic
        nb_actions :      array(int), optional
                          Numbers of rating to be taken for each sample made on user's historic
        
        Returns
        -------
        states :         List(String)
                        All the states sampled, format of a sample: itemId&rating
        actions :        List(String)
                        All the actions sampled, format of a sample: itemId&rating
      

        Notes
        -----
        States must be before(timestamp<) the actions.
        If given, size of nb_states is the numbller of sample by user
        sizes of nb_states and nb_actions must be equals
        '''

        n = len(user_histo)
        sep = int(action_ratio * n)
        nb_sample=1
        # nb_sample = random.randint(1, max_samp_by_user)
        if not nb_states:
            nb_states = [sep for i in range(nb_sample)]
        if not nb_actions:
            nb_actions = [n-sep for i in range(nb_sample)]
        assert len(nb_states) == len(nb_actions), 'Given array must have the same size'
        
        states  = []
        actions = []
        
        # Select samples in histo 
        for i in range(len(nb_states)):
            sample_states = user_histo.iloc[0:sep]
            
            sample_actions = user_histo.iloc[-(n - sep):]
            
            sample_state  = []
            sample_action = []
            
            for j in range(nb_states[i]):
                row = sample_states.iloc[j]
                # Formate State
                state = row.loc['track_id'] #TODO: REWARD SHAPING
                sample_state.append(state)
          
            for j in range(nb_actions[i]):
                row = sample_actions.iloc[j]
                # Formate Action 
                action = row.loc['track_id']
                sample_action.append(action)
            

            states.append(sample_state)
            actions.append(sample_action)
                
        return sample_state, sample_action

In [40]:
user_state, action, response =sample_histo(self = None, user_histo = historic_users[0], action_ratio = 0.25, max_samp_by_user = 10,  max_state = 1, max_action = 15,
                 nb_states = [], nb_actions = [])
print(user_state)
print(action)
print(response)

[21974, 980, 12827, 4176, 17890]
[18654, 12649, 20837, 850, 1424, 32211, 10888, 2705, 9203, 15250, 850, 25667, 4371, 22348, 234]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
music_columns = ['acousticness','beat_strength', 'bounciness', 'danceability', 
                 'energy', 'liveness', 'speechiness', 'valence']

In [21]:
context_feature_vectors = {}

# Loop through the track_ids and collect the values for selected columns
for track_id in user_state:
    context_features = historic_users[0].loc[historic_users[0]['track_id'] == track_id, music_columns].values[0]
    context_feature_vectors[track_id] = context_features

# Print the context feature vectors
for track_id, context_vector in context_feature_vectors.items():
    print(f"Track ID: {track_id}, Context Feature Vector: {context_vector}")

Track ID: 21974, Context Feature Vector: [0.13176584 0.4433504  0.42268586 0.59695691 0.66006583 0.0947587
 0.03311838 0.74237645]
Track ID: 980, Context Feature Vector: [0.39685428 0.40093413 0.35999015 0.55222732 0.51458508 0.36092412
 0.03833732 0.85907483]
Track ID: 12827, Context Feature Vector: [0.71783501 0.33719796 0.33358982 0.39221773 0.62889779 0.12860203
 0.03349711 0.72008985]
Track ID: 4176, Context Feature Vector: [0.32564566 0.37090713 0.37293941 0.39572051 0.47290361 0.10919463
 0.03447702 0.38472331]
Track ID: 17890, Context Feature Vector: [0.6608218  0.57611763 0.57833576 0.55020553 0.85367286 0.11169621
 0.03599762 0.96541512]


In [23]:
context_feature_vectors=np.array(context_feature_vectors)

In [24]:
context_feature_vectors

array({21974: array([0.13176584, 0.4433504 , 0.42268586, 0.59695691, 0.66006583,
       0.0947587 , 0.03311838, 0.74237645]), 980: array([0.39685428, 0.40093413, 0.35999015, 0.55222732, 0.51458508,
       0.36092412, 0.03833732, 0.85907483]), 12827: array([0.71783501, 0.33719796, 0.33358982, 0.39221773, 0.62889779,
       0.12860203, 0.03349711, 0.72008985]), 4176: array([0.32564566, 0.37090713, 0.37293941, 0.39572051, 0.47290361,
       0.10919463, 0.03447702, 0.38472331]), 17890: array([0.6608218 , 0.57611763, 0.57833576, 0.55020553, 0.85367286,
       0.11169621, 0.03599762, 0.96541512])}, dtype=object)

In [32]:
music_columns = ['acousticness','beat_strength', 'bounciness', 'danceability', 
                 'energy', 'liveness', 'speechiness', 'valence']
sequence_feature_vectors = {}

# Loop through the track_ids and collect the values for selected columns
for track_id in action:
    sequence_features = historic_users[0].loc[historic_users[0]['track_id'] == track_id, music_columns].values[0]
    sequence_feature_vectors[track_id] = sequence_features

# Print the context feature vectors
for track_id, sequence_vector in sequence_feature_vectors.items():
    print(f"Track ID: {track_id}, Context Feature Vector: {sequence_vector}")
sequence_feature_vectors=np.array(sequence_feature_vectors)

Track ID: 18654, Context Feature Vector: [0.2533223  0.37939456 0.38062477 0.52122897 0.48679543 0.05788048
 0.0299142  0.48322469]
Track ID: 12649, Context Feature Vector: [0.36027166 0.38219875 0.39523438 0.38951439 0.62458581 0.06625892
 0.053722   0.51449978]
Track ID: 20837, Context Feature Vector: [0.01027942 0.46818495 0.44707748 0.68270832 0.63663459 0.24463625
 0.03197116 0.93102282]
Track ID: 850, Context Feature Vector: [0.65055788 0.43855327 0.38985005 0.5219034  0.3704311  0.14220387
 0.02526775 0.65329283]
Track ID: 1424, Context Feature Vector: [0.18433571 0.42684042 0.36469597 0.34078264 0.41622573 0.09168113
 0.026546   0.49927628]
Track ID: 32211, Context Feature Vector: [0.63585979 0.33515647 0.31789699 0.38874629 0.58511174 0.17361492
 0.03810181 0.961348  ]
Track ID: 10888, Context Feature Vector: [0.63130581 0.40154979 0.37268782 0.45972061 0.54624927 0.1567937
 0.02880025 0.85192317]
Track ID: 2705, Context Feature Vector: [0.63779247 0.60341609 0.6195659  0.7304

In [33]:
sequence_feature_vectors

array({18654: array([0.2533223 , 0.37939456, 0.38062477, 0.52122897, 0.48679543,
       0.05788048, 0.0299142 , 0.48322469]), 12649: array([0.36027166, 0.38219875, 0.39523438, 0.38951439, 0.62458581,
       0.06625892, 0.053722  , 0.51449978]), 20837: array([0.01027942, 0.46818495, 0.44707748, 0.68270832, 0.63663459,
       0.24463625, 0.03197116, 0.93102282]), 850: array([0.65055788, 0.43855327, 0.38985005, 0.5219034 , 0.3704311 ,
       0.14220387, 0.02526775, 0.65329283]), 1424: array([0.18433571, 0.42684042, 0.36469597, 0.34078264, 0.41622573,
       0.09168113, 0.026546  , 0.49927628]), 32211: array([0.63585979, 0.33515647, 0.31789699, 0.38874629, 0.58511174,
       0.17361492, 0.03810181, 0.961348  ]), 10888: array([0.63130581, 0.40154979, 0.37268782, 0.45972061, 0.54624927,
       0.1567937 , 0.02880025, 0.85192317]), 2705: array([0.63779247, 0.60341609, 0.6195659 , 0.73043108, 0.69299668,
       0.17496482, 0.03296616, 0.9648841 ]), 9203: array([0.22108336, 0.43417022, 0.363684

In [44]:
response_columns = ['skip_2']
response_feature_vectors = {}

# Loop through the track_ids and collect the values for selected columns
for track_id in action:
    response_features = historic_users[0].loc[historic_users[0]['track_id'] == track_id, response_columns].values[0]
    response_feature_vectors[track_id] = response_features

# Print the context feature vectors
for track_id, response_vector in response_feature_vectors.items():
    print(f"Track ID: {track_id}, Context Feature Vector: {response_vector}")
response_feature_vectors=np.array(response_feature_vectors)

Track ID: 18654, Context Feature Vector: [0]
Track ID: 12649, Context Feature Vector: [0]
Track ID: 20837, Context Feature Vector: [1]
Track ID: 850, Context Feature Vector: [0]
Track ID: 1424, Context Feature Vector: [0]
Track ID: 32211, Context Feature Vector: [0]
Track ID: 10888, Context Feature Vector: [0]
Track ID: 2705, Context Feature Vector: [0]
Track ID: 9203, Context Feature Vector: [0]
Track ID: 15250, Context Feature Vector: [0]
Track ID: 25667, Context Feature Vector: [0]
Track ID: 4371, Context Feature Vector: [0]
Track ID: 22348, Context Feature Vector: [0]
Track ID: 234, Context Feature Vector: [0]


In [45]:
response_feature_vectors

array({18654: array([0]), 12649: array([0]), 20837: array([1]), 850: array([0]), 1424: array([0]), 32211: array([0]), 10888: array([0]), 2705: array([0]), 9203: array([0]), 15250: array([0]), 25667: array([0]), 4371: array([0]), 22348: array([0]), 234: array([0])},
      dtype=object)

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Sample data (replace with your actual data)
context_feature_vectors = {
    21974: [0.13176584, 0.4433504, 0.42268586, 0.59695691, 0.66006583, 0.0947587, 0.03311838, 0.74237645],
    980: [0.39685428, 0.40093413, 0.35999015, 0.55222732, 0.51458508, 0.36092412, 0.03833732, 0.85907483],
    # Add other samples
}

track_features = {
    18654: [0.2533223, 0.37939456, 0.38062477, 0.52122897, 0.48679543, 0.05788048, 0.0299142, 0.48322469],
    12649: [0.36027166, 0.38219875, 0.39523438, 0.38951439, 0.62458581, 0.06625892, 0.053722, 0.51449978],
    # Add other samples
}

user_responses = {
    18654: [0],
    12649: [0],
    # Add other samples
}

# Combine data into sequences
sequences = []
for track_id in user_responses:
    if track_id in context_feature_vectors and track_id in track_features:
        context_vector = context_feature_vectors[track_id]
        track_feature = track_features[track_id]
        user_response = user_responses[track_id]
        sequences.append((context_vector, track_feature, user_response))

# Convert data to PyTorch tensors
X_context = torch.tensor([sequence[0] for sequence in sequences], dtype=torch.float32)
X_track = torch.tensor([sequence[1] for sequence in sequences], dtype=torch.float32)
y = torch.tensor([sequence[2] for sequence in sequences], dtype=torch.float32)

# Define a custom dataset for PyTorch
class CustomDataset(Dataset):
    def __init__(self, X_context, X_track, y):
        self.X_context = X_context
        self.X_track = X_track
        self.y = y

    def __len__(self):
        return len(self.X_context)

    def __getitem__(self, idx):
        return self.X_context[idx], self.X_track[idx], self.y[idx]

# Create DataLoader
batch_size = 32
dataset = CustomDataset(X_context, X_track, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the LSTM model
class SequentialLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SequentialLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x_context, x_track):
        combined_features = torch.cat((x_context, x_track), dim=1)
        lstm_out, _ = self.lstm(combined_features)
        out = self.fc(lstm_out)
        return out

# Initialize the model with (500, 200, 200) units
input_size = X_context.shape[1] + X_track.shape[1]  # Combine input sizes
hidden_size = 500
num_layers = 3
output_size = 1  # Binary classification

model = SequentialLSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with sequence prediction
num_epochs = 10
for epoch in range(num_epochs):
    hidden = None  # Initialize hidden state for the sequence prediction
    for context, track, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(context, track)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save or use the trained model as needed


ValueError: libcublas.so.*[0-9] not found in the system path ['/home/aayush/rl_spotifyrecs/src/rl_spotifyrecs/spotify_data', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/aayush/rl_spotifyrecs/.venv/lib/python3.11/site-packages']